# Import

In [1]:
import numpy as np
import pandas as pd
import csv
import datetime
import math

# myLibrary

In [2]:
#製造answer.csv的第1列
def answer_header():
    header = ['user_id']
    for i in range(28):
        header.append('time_slot_' + str(i) )
    return header

In [3]:
def get_week(date_str):
    d = get_date(date_str)
    w = datetime.datetime.strftime(d, '%w')
    return int(w)

In [4]:
def get_date(date_str):
    return datetime.datetime.strptime(date_str, '%Y-%m-%d')

In [5]:
def get_slot(time_str):
    the_time = datetime.datetime.strptime(time_str, '%H:%M:%S.%f')
    time_sec0 = datetime.datetime.strptime('01:00:00.00', '%H:%M:%S.%f')
    time_sec1 = datetime.datetime.strptime('09:00:00.00', '%H:%M:%S.%f')
    time_sec2 = datetime.datetime.strptime('17:00:00.00', '%H:%M:%S.%f')
    time_sec3 = datetime.datetime.strptime('21:00:00.00', '%H:%M:%S.%f')
    time_sec4 = datetime.datetime.strptime('23:59:59.99', '%H:%M:%S.%f')
    if time_sec0 <= the_time and the_time <= time_sec1:
        slot = 0
    elif time_sec1 <= the_time and the_time <= time_sec2:
        slot = 1
    elif time_sec2 <= the_time and the_time <= time_sec3:
        slot = 2
    elif time_sec3 <= the_time and the_time <= time_sec4:
        slot = 3
    else:
        slot = 4
    return slot

In [6]:
def get_days(end, start):
    d = end - start
    return d.days

In [7]:
def update_week(week, date_str, event_week, addition):
    date = get_date(date_str)
    end_date = get_date('2017-08-14')
    days = get_days(end_date, date)
    weight = 0
    for i in range(1,33):
        if (days/7) <= i:
            weight = math.pow(0.832, i)
            break
    addition = list(np.array(addition) * weight)
    week.days[event_week].add_slots(addition)
    another_addition = [0, 0, 0]
    another_addition.append(addition[4])
    week.days[event_week-1].add_slots(another_addition)
    return week

# myClass

In [8]:
class Day():
    def __init__(self):
        self.slot0, self.slot1, self.slot2, self.slot3 = 0, 0, 0, 0
    def reset(self):
        self.slot0, self.slot1, self.slot2, self.slot3 = 0, 0, 0, 0
    def get_slots(self):
        return [self.slot0, self.slot1, self.slot2, self.slot3]
    def add_slots(self, addition):
        self.slot0 += addition[0]
        self.slot1 += addition[1]
        self.slot2 += addition[2]
        self.slot3 += addition[3]

In [9]:
class Week():
    def __init__(self):
        self.days = []
        for i in range(7):
            self.days.append(Day())
    def reset(self):
        for i in range(7):
            self.days[i].reset()
    def get_answer(self, id):
        tmp = []
        for i in range(1, 7):
            tmp += self.days[i].get_slots()
        tmp += self.days[0].get_slots()
        answer = [id] + list(np.array(tmp))
        return answer

# Main

In [10]:
with open('answer.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(answer_header())
    week = Week()
    # 開始處理資料
    for data_count in range(46, 76): # revise
        data = pd.read_csv('private/data-0' + str(data_count) + '.csv')
        
        # 取得目前csv檔案的首尾user_id
        head = data.head(1)['user_id'].values[0]
        tail = data.tail(1)['user_id'].values[0]
        
        # 依序處理各id資料並寫入answer.csv
        for id in range(head, tail+1):
            id_data = data[data['user_id'] == id]
            length = len(id_data)
            
            # 依序處理某id的每一筆資料
            now_date = id_data['event_time'].values[0].split()[0]
            addition = [0, 0, 0, 0, 0]
            for i in range(length):
                # 切分時間獲得星期和slot
                event_time = id_data['event_time'].values[i].split()
                date = event_time[0]
                
                # 當日期變化
                if date != now_date:
                    # 將前日的紀錄更新進week
                    event_week = get_week(now_date)
                    week = update_week(week, now_date, event_week, addition)
                    #並初始化now_date和addition
                    now_date = date
                    addition = [0, 0, 0, 0, 0]
                
                # 紀錄活動至addition
                event_slot = get_slot(event_time[1])
                addition[event_slot] = 1
            
            # 補齊最後一段日期的紀錄
            event_week = get_week(now_date)
            week = update_week(week, now_date, event_week, addition)
            
            # 寫入answer.csv
            writer.writerow(week.get_answer(id))
            
            # 初始化week
            week.reset()
            
            #break # revise

# test

# Test Area

In [11]:
id_data = data[data['user_id'] == 57159]
id_data

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer


In [12]:
data

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer
0,58396,117520,5735060,75,2017-02-03 16:05:04.412,187,seek,Android,1,18,cellular 4G,False
1,58396,117520,5735060,75,2017-02-03 16:05:13.158,6,seek,Android,1,18,cellular 4G,False
2,58396,117520,5735060,75,2017-02-03 16:05:42.592,28,seek,Android,1,18,cellular 4G,False
3,58396,117520,5735060,75,2017-02-03 16:05:51.241,8,leave,Android,1,18,cellular 4G,False
4,58396,117520,5735060,148,2017-02-03 16:07:40.547,2,seek,Android,6,20,cellular 4G,False
5,58396,117520,5735060,148,2017-02-03 16:13:34.177,352,seek,Android,6,20,cellular 4G,False
6,58396,117520,5735060,148,2017-02-03 16:13:37.620,2,seek,Android,6,20,cellular 4G,False
7,58396,117520,5735060,148,2017-02-03 16:13:39.987,1,seek,Android,6,20,cellular 4G,False
8,58396,117520,5735060,148,2017-02-03 16:13:46.658,5,seek,Android,6,20,cellular 4G,False
9,58396,117520,5735060,148,2017-02-03 16:14:01.952,14,seek,Android,6,20,cellular 4G,False


# Check Answer

In [13]:
answer = pd.read_csv('answer.csv')
answer

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,57159,3.432906,3.444926,1.592012,2.622720,3.999278,4.277842,2.634528,3.102281,3.084284,...,2.015615,1.326734,1.566175,1.040592,0.561418,1.898587,2.213828,3.464402,2.534487,3.550688
1,57160,0.331696,1.855920,0.331696,0.000000,0.730369,2.254593,0.331696,0.000000,2.138299,...,0.607667,0.000000,0.000000,0.974603,0.331696,0.000000,0.000000,1.183597,0.968195,0.000000
2,57161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.575930,0.575930,0.000000,0.692224,0.000000,0.000000,0.000000,0.000000
3,57162,0.000000,0.000000,0.000000,0.000000,0.000000,0.832000,0.000000,0.000000,0.000000,...,1.743277,0.000000,0.158940,0.518231,1.040961,0.000000,0.110022,0.685952,0.575930,0.000000
4,57163,0.692224,0.692224,0.000000,0.000000,0.692224,0.692224,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,57164,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.692224,0.000000,0.000000,0.000000,0.575930,0.000000,0.575930,0.000000,0.000000
6,57165,0.832000,1.992844,0.734870,0.317880,0.000000,3.345935,1.544125,1.820974,0.000000,...,0.158940,0.000000,1.330338,0.755145,0.692224,0.229608,1.054070,1.676363,0.692224,1.100433
7,57166,0.000000,1.524224,0.000000,0.000000,0.000000,1.524224,0.000000,0.000000,0.000000,...,0.692224,0.000000,0.000000,0.728718,0.000000,0.000000,0.575930,1.323779,0.692224,0.000000
8,57167,1.723059,3.266386,0.965560,0.000000,3.194007,4.028287,0.496857,0.000000,3.046572,...,1.331024,0.000000,1.614259,2.426277,0.769376,0.132238,2.636723,3.058980,0.537912,0.000000
9,57168,3.645014,4.308243,0.000000,4.132111,4.417644,4.011808,0.000000,5.489958,4.330232,...,0.000000,0.434828,2.034041,0.960989,0.000000,1.260291,1.576810,0.122440,0.000000,3.886620
